### The code for data cleaning

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re

df = pd.read_csv('listings.csv', encoding='utf-8') #read the Airbnb listing data
print('The size of original dataset is (%s,%s).'%(df.shape[0],df.shape[1]))

#---------------------------------------------------------------------------------------------------------------------------------------#
###### Step 1: Data cleaning based on the data
#checking for any duplicate data by id
print(len(df[df['id'].duplicated()])) #--no duplicate data

#delete data without location detail
df1 = df[~df['longitude'].isna()]
df1 = df1[~df1['latitude'].isna()]
print('After deleting data without location, the size of dataset is (%s,%s).'%(df1.shape[0],df1.shape[1]))

#delete data with the value of accommodates is 0
df1 = df1[df1['accommodates']>0]
print('After deleting data with 0 accommodate, the size of dataset is (%s,%s).'%(df1.shape[0],df1.shape[1]))

# delete the record which have no review
df1 = df1[df1['number_of_reviews']>0]
print('After deleting data with 0 review, the size of dataset is (%s,%s).'%(df1.shape[0],df1.shape[1]))

# delete outlier which the value of minimum_nights is too high (to conserve listings for short-term rental)
df2 = df1[df1['minimum_nights']<=7]
print('After deleting listings with a minimum stay of 8 nights, the size of dataset is (%s,%s).'%(df2.shape[0],df2.shape[1]))

# delete the missing value
#checking for null value
pd.DataFrame(df2.isnull().sum(), columns = ['Count Null'])
# delete the listings without a bed or a bedroom
df3 = df2[~(df2['beds'].isna()&df2['bedrooms'].isna())]
print('After deleting listings without a bed or a bedroom, the size of dataset is (%s,%s).'%(df3.shape[0],df3.shape[1]))
#checking for null value
pd.DataFrame(df3.isnull().sum(), columns = ['Count Null'])
# delete the listings without description or neighbourhood_overview
df4 = df3[~(df3['description'].isnull()&df3['neighborhood_overview'].isnull())]
print('After deleting listings without description or neighbourhood_overview, the size of dataset is (%s,%s).'%(df4.shape[0],df4.shape[1]))
#checking for null value
pd.DataFrame(df4.isnull().sum(), columns = ['Count Null'])

# reorganize the dataset
airbnb = df4.reset_index(drop=True)
airbnb['neighbourhood_cleansed'] = airbnb['neighbourhood_cleansed'].str.split('/ ',expand = True)[1]
airbnb['neighbourhood_cleansed'] = [i.replace(' District', '') for i in airbnb['neighbourhood_cleansed']]
# Convert the date to datetime64
airbnb['last_scraped'] = pd.to_datetime(airbnb['last_scraped'], format='%Y-%m-%d')
airbnb['host_since'] = pd.to_datetime(airbnb['host_since'], format='%Y-%m-%d')
#Convert the data type of price
airbnb['price'] = [float(i.replace('$', '').replace(',', '')) for i in airbnb['price']]

#Combine description and neighbourhood_overview together
airbnb['description_all'] = 41
for i in range(len(airbnb)):
    if airbnb.loc[i,'description'] == "":
        airbnb.loc[i,'description_all'] = airbnb.loc[i,'neighborhood_overview']
    if (~(airbnb.loc[i,'description']=="") and (airbnb.loc[i,'neighborhood_overview'] == "")):
        airbnb.loc[i,'description_all'] = airbnb.loc[i,'description']
    if (~(airbnb.loc[i,'description']=="") and (~(airbnb.loc[i,'neighborhood_overview'] == ""))):
        airbnb.loc[i,'description_all'] = str(airbnb.loc[i,'description']) +" " +str(airbnb.loc[i,'neighborhood_overview'])
    #remove nan
    if airbnb.loc[i,'description_all'][:4] == "nan ":
        airbnb.loc[i,'description_all'] =  airbnb.loc[i,'description_all'][4:]
    if airbnb.loc[i,'description_all'][-4:] == " nan":
        airbnb.loc[i,'description_all'] =  airbnb.loc[i,'description_all'][:-4]

airbnb = pd.DataFrame(airbnb, columns = ['id', 'last_scraped', 'latitude', 'longitude','name', 'description_all',
                                          'host_id','host_since', 'calculated_host_listings_count',
                                          'neighbourhood_cleansed', 'property_type','room_type',
                                          'accommodates', 'bathrooms_text','bedrooms', 'beds','price',
                                          'minimum_nights', 'number_of_reviews','reviews_per_month', 
                                          'review_scores_rating','review_scores_location']) 
airbnb.rename(columns = {"description_all":"description"}, inplace = True)
airbnb.rename(columns = {"neighbourhood_cleansed":"district"}, inplace = True)

#checking for null value
pd.DataFrame(airbnb.isnull().sum(), columns = ['Count Null'])
print('The size of final dataset in this step is (%s,%s).'%(airbnb.shape[0],airbnb.shape[1]))

#---------------------------------------------------------------------------------------------------------------------------------------#
###### Step 2: Data cleaning based on the location
# City boundry
zone = gpd.read_file(r'boundary/县.shp')
SH_z = zone[zone['市']=='上海市']
SH_z = SH_z.to_crs(4490)

# airbnb distribution
airbnb_g = gpd.GeoDataFrame(airbnb,geometry = gpd.points_from_xy(airbnb.longitude,airbnb.latitude),crs = 'EPSG:4326')
airbnb_g = airbnb_g.to_crs(4490)

a=['Pudong', 'Huangpu', 'Xuhui', 'Jing\'an', 'Changning', 'Hongkou', 'Yangpu', 'Putuo','Minhang','Baoshan','Jiading','Jinshan','Songjiang','Qingpu','Fengxian','Chongming']
c=['浦东新区', '黄浦区', '徐汇区', '静安区', '长宁区', '虹口区', '杨浦区', '普陀区','闵行区','宝山区','嘉定区','金山区','松江区','青浦区','奉贤区','崇明区']
d={"name" : a,
   "district_zh" : c}
district = pd.DataFrame(d)

# delete the listings which located outside its district
datalist = []
for i in range(16):
    df = pd.DataFrame()
    datalist.append(df)

def data_sub(name):
    df1 = district[district['name']==name]
    df1 = df1.reset_index(drop=True)
    district_en =df1.loc[0,'name']
    district_zh =df1.loc[0,'district_zh']
    airbnb_sub = airbnb_g[airbnb_g['district'] == district_en]
    SH_z_sub = SH_z[SH_z['NAME'] == district_zh]
    return airbnb_sub, SH_z_sub

if __name__ == '__main__':
    for i in range(len(district)):
        s = district.loc[i,'name']
        airbnb_sub, SH_z_sub = data_sub(s)
        datalist[i] = gpd.sjoin(airbnb_sub,SH_z_sub,how = "inner")
        datalist[i] = pd.DataFrame(datalist[i], columns = ['id', 'last_scraped', 'latitude', 'longitude','name', 'description',
                                          'host_id','host_since', 'calculated_host_listings_count',
                                          'district', 'property_type','room_type',
                                          'accommodates', 'bathrooms_text','bedrooms', 'beds','price',
                                          'minimum_nights', 'number_of_reviews','reviews_per_month', 
                                          'review_scores_rating','review_scores_location']) 
        

    airbnb_2 = pd.concat(datalist).reset_index(drop=True)
    print('After deleting listings located outside its district, the size of dataset is (%s,%s).'%(airbnb_2.shape[0],airbnb_2.shape[1]))

#---------------------------------------------------------------------------------------------------------------------------------------#
###### Step 3: Data cleaning based on the description
# reference: https://blog.csdn.net/sinat_36972314/article/details/79746291
airbnb_2['description_en'] = 21 # new column to store the English description
airbnb_2['description_zh'] = 31 # new column to store the Chinese description

def replace_useless(sentence):
    sentence = re.sub('\s+', ' ', sentence)  # remove newline chars
    sentence = re.sub("<b>", " ", sentence)  # remove <b>
    sentence = re.sub("<br />", " ", sentence)  # remove <br />
    sentence = re.sub("</b>", " ", sentence) # remove </b>
    sentence = re.sub("The space ", " ", sentence) # remove The space 
    sentence = re.sub("Other things to note ", " ", sentence) # remove Other things to note
    sentence = re.sub("Guest access ", " ", sentence) # remove Guest access 
    sentence = re.sub(',', ' ', sentence)  # remove ,        
    return sentence        

for i in range(len(airbnb_2)):
    airbnb_2.loc[i,'description'] = replace_useless(airbnb_2.loc[i,'description'])
    uncn = re.compile(r'[\u0061-\u007a, \u0020]')
    airbnb_2.loc[i,'description_en'] = "".join(uncn.findall(airbnb_2.loc[i,'description'].lower()))
    uncn = re.compile(r'[\u4e00-\u9fa5]')
    airbnb_2.loc[i,'description_zh'] = "".join(uncn.findall(airbnb_2.loc[i,'description'].lower()))

airbnb_2['count_en'] = airbnb_2['description_en'].apply(lambda n: len(n.split()))
airbnb_2['count_zh'] = airbnb_2['description_zh'].apply(lambda n: len(n))

print('The size of data which have no English or Chinese description: %s.'%(len(airbnb_2[(airbnb_2['count_zh']== 0)&(airbnb_2['count_en']== 0)])))
#only Chinese description
print('The size of data which have no English description: %s.'%len(airbnb_2[airbnb_2['count_en']== 0]))
#only English description
print('The size of data which have no Chinese description: %s.'%len(airbnb_2[airbnb_2['count_zh']== 0]))

#mix language description
print('The size of data which have two languages description: %s.'%(len(airbnb_2[(airbnb_2['count_zh']>0) & (airbnb_2['count_en']>0)])))

# reselect the data
airbnb_3 = airbnb_2[airbnb_2['count_zh']>0] # Chinses part

airbnb_3 = airbnb_3.reset_index(drop=True)
print('After deleting listings without Chinese description, the size of dataset is (%s,%s).'%(airbnb_3.shape[0],airbnb_3.shape[1]))

# Delete data whose Chinese description is less than 1 percentile words long and more than 99 percentile words long.      
a = airbnb_3['count_zh'].quantile(1-0.01)
b = airbnb_3['count_zh'].quantile(0.01)

airbnb_3 = airbnb_3[((airbnb_3['count_zh']<=a)&(airbnb_3['count_zh']>=b))]
airbnb_3 = airbnb_3.reset_index(drop=True)

# Final dataset to next step
print('The size of final dataset is (%s,%s).'%(airbnb_3.shape[0],airbnb_3.shape[1]))

# Export cleaned data for next step
airbnb_3.to_csv("cleaned data airbnb.csv", index=False, index_label=False) 